<a href="https://colab.research.google.com/github/Abhishekravindran/TF_LITE-OPTIMIZATION-For-DEEP-NEURAL-NET/blob/main/Cnn_Weight_optimization_tf_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets Try to implement weight Size Allocated in memory by a normal keras dnn by few process as below

In [ ]:
# importing all the necessary libraries
import os
import tensorflow as tf
import h5py
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sys import getsizeof

In [ ]:
print(tf.__version__)

In [ ]:
#Fun to caluclate size
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:
#fun to convert a given size into bytes
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
# loading the mnist dataset on which we will see the optimization
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train.shape,x_test.shape

In [ ]:
#vizualizing
import matplotlib.pyplot as plt
%matplotlib inline
i=np.random.randint(0,59999)
plt.imshow(x_train[i],cmap='gray')
y_train[i]

In [ ]:
#vizualizing
width=12
height=12
fig,axes=plt.subplots(width,height,figsize=(15,15))
axes=axes.ravel() #10*10 =100 converting matrix to vectors
for i in np.arange(0,width*height):
  index=np.random.randint(0,59999)
  axes[i].imshow(x_train[index],cmap='gray');
  axes[i].axis('off')
  axes[i].set_title(y_train[index],fontsize=10)
plt.subplots_adjust(hspace=0.8)

In [ ]:
#preprocessing
x_train = x_train / 255.0
x_test = x_test / 255.0

Building the model and compiling and doing a fit over the train data

In [ ]:

model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

In [ ]:
model.compile(optimizer='adam',
              loss= SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
KERAS_MODEL_NAME = "tf_digit_mnist.h5"

In [ ]:
# saving the model
model.save(KERAS_MODEL_NAME)

In [ ]:
#now as we can see the size of he file please do keep a note on how we will optimize this without affecting the accuracy
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")

In [ ]:
#keep the note of accuracy for further cross check
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Creating a Tf_lite MOdel

In [ ]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [ ]:
# if we have a specific optimization size we can manually give it else directly convert it to min optimium size using the optimizations
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
#as we can see how we have reduced the storage size from the previous result of keras model 
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Checking **Input** Tensor Shape

In [ ]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Resize the Tensor Shape

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 28, 28))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
x_test.dtype

In [ ]:
x_test_numpy = np.array(x_test, dtype=np.float32)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], x_test_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
acc = accuracy_score(prediction_classes, y_test)

In [ ]:
# we can also notice that our model hasnt got tampred with the accuracy much comparitively w.r.t what we had obtained previously
print('Test accuracy TFLITE model :', acc)

Access Quantized Weights of Keras model

In [ ]:
#Lets see how internally we are saving memory so lets randomly take a single intialized weight and check the size occupied
print(model.get_weights()[0][0][0], type(model.get_weights()[0][0][0]))

In [ ]:
keras_weight_var = np.array([model.get_weights()[0][0][0]], dtype="float64")

In [ ]:
print("The weight value occupied without optimization is around {} bytes in the memory".format(getsizeof(keras_weight_var[0]))) 

In order to access the tflite model weights have to load the model and read the corresponding values form the layers and  save it into an h5 file 

In [ ]:
TF_LITE_WEIGHTS_TEMP_FILE = "temp_weights_from_tflite.h5"

In [ ]:
interpreter = tf.lite.Interpreter(model_path=TF_LITE_MODEL_FILE_NAME)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
all_layers_details = interpreter.get_tensor_details() 
f = h5py.File(TF_LITE_WEIGHTS_TEMP_FILE, "w")   
for layer in all_layers_details:
     grp = f.create_group(str(layer['index']))
     grp.attrs["name"] = layer['name']
     grp.attrs["shape"] = layer['shape']
     grp.attrs["quantization"] = layer['quantization']
     grp.create_dataset("weights", data=interpreter.get_tensor(layer['index']))
f.close()

Getting optimized weights of Tflite model

In [ ]:
#randomly loading the weight frm our tflite model
temp_file = h5py.File(TF_LITE_WEIGHTS_TEMP_FILE, 'r')
print(temp_file["5"]["weights"][0][6], type(temp_file["5"]["weights"][0][6]))

In [ ]:
quantized_weight_var = np.array([temp_file["5"]["weights"][0][6]])

In [ ]:
print("The weight value occupied with optimization is around {} bytes in the memory".format(getsizeof(quantized_weight_var[0])))

Conclusion as we can see for a single weight there is a difference of 7 bytes imagine a huge deep neural network where in there are 10kk+ neurons then the change/size occupied is enormous and however in scenarios like where we have to implement our dnn model in android mobile phones eg: image detection we cannot afford to provide memory space so high hence we do this optimiztaion for similar working and required accuracy

BElow is the same implementation of optimization on fashion_mnist dataset  and the conclusion remains the same as above

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Explore the data

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
np.unique(train_labels)

# Test Dataset

In [ ]:
test_images.shape

In [ ]:
len(test_labels)

## Preprocessing

In [ ]:
plt.figure()
plt.imshow(train_images[88])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## Build & Compile the model


In [ ]:
model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

In [ ]:
model.compile(optimizer='adam',
              loss= SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=10)

In [ ]:
KERAS_MODEL_NAME = "tf_model_fashion_mnist.h5"

In [ ]:
model.save(KERAS_MODEL_NAME)

In [ ]:
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

# TF Lite Model

In [ ]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

# Check Input Tensor Shape

In [ ]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

# Resize Tensor Shape

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 28, 28))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
test_images.dtype

In [ ]:
test_imgs_numpy = np.array(test_images, dtype=np.float32)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
acc = accuracy_score(prediction_classes, test_labels)

In [ ]:
print('Test accuracy TFLITE model :', acc)

# Get Weights of Keras Model

In [ ]:
print(model.get_weights()[0][0][0], type(model.get_weights()[0][0][0]))

In [ ]:
keras_weight_var = np.array([model.get_weights()[0][0][0]], dtype="float64")

In [ ]:
getsizeof(keras_weight_var[0])

# Access Quantized Weights of TFLite

In [ ]:
TF_LITE_WEIGHTS_TEMP_FILE = "temp_weights_from_tflite.h5"

In [ ]:
interpreter = tf.lite.Interpreter(model_path=TF_LITE_MODEL_FILE_NAME)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
all_layers_details = interpreter.get_tensor_details() 
f = h5py.File(TF_LITE_WEIGHTS_TEMP_FILE, "w")   
for layer in all_layers_details:
     grp = f.create_group(str(layer['index']))
     grp.attrs["name"] = layer['name']
     grp.attrs["shape"] = layer['shape']
     grp.attrs["quantization"] = layer['quantization']
     grp.create_dataset("weights", data=interpreter.get_tensor(layer['index']))
f.close()

In [ ]:
temp_file = h5py.File(TF_LITE_WEIGHTS_TEMP_FILE, 'r')
print(temp_file["5"]["weights"][0][6], type(temp_file["5"]["weights"][0][6]))

In [ ]:
quantized_weight_var = np.array([temp_file["5"]["weights"][0][6]])

In [ ]:
getsizeof(quantized_weight_var[0])

In [ ]:
temp_file.close()